![Semillero Astrofisica](../logo.PNG)

---
# Ecuaciones Diferenciales Ordinarias: Introducción 


Carlos Andrés del Valle (cdelv@unal.edu.co)

---

- ODE de 2 orden normal
- ODE con términos de orden superior
- Sistema de ODEs
- ODE estocástica
- ODE integrodiferencial
- ODE cond de frontera

- ODE inversa normal